In [ ]:
#install methods
!pip install transformers

import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
import pandas as pd
import numpy as np
import time
import pprint
import sys

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print("Belangrijke libraries zijn geimporteerd.")

In [ ]:
#load data for creating word embeddings

i=0
reviews=[]
file = open('Desktop/1restDataSome.txt', 'r')
review = file.read()
review = review.replace("\n", ' ')

#Reviews are split on ,|,
wrong_review = review.split(",|,") 
for w in wrong_review:
    i += 1
    reviews.append(w)
print("Number of reviews : ",len(reviews))

In [ ]:
#Code for removing negation words in same sentence
i = 0
review = 0
sentC=0
reviewsNew = []
neg = " not "
neg2 = " nothing "
neg3 = " never "
neg4 = " didn\'t "
neg5 = " wouldn\'t "
neg6 = " don\'t "
neg7 = " can\'t "
neg8 = " doesn\'t "
neg9 = " couldn\'t "
case=0
for z in reviews:
    sent2New = []
    sentNew= ""
    sent = z.split('.')
    case = 0
    for j in sent:
        sent2 = j.split('!')
        for l in sent2:
            sentC+=1
            if (neg in l) or (neg2 in l) or (neg3 in l) or (neg4 in l) or (neg5 in l) or (neg6 in l) or (neg7 in l) or (neg8 in l) or (neg9 in l):
                i += 1
                sent2.remove(l)
                if case == 0:
                    review += 1
                    case = 1
        sent2New.append("!".join(sent2))
    #print(sent2New)
    sentNew = (".".join(sent2New))
    reviewsNew.append(str(sentNew))

print("Number of sentences with negation word:", i)
print("Number of reviews with these negation word:" ,review)
print("Number of sentences in text: ", sentC)
print("Number of sentences in text after removing sentences: ", sentC - i )

In [ ]:

# Load pre-trained model (weights)

model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True,)                    #pretrained
#model = BertModel.from_pretrained('/Users/markrademaker/Desktop/PostBERTModel100K',output_hidden_states = True,)                    #posttrained
#model = BertModel.from_pretrained('/Users/markrademaker/Desktop/BERT_FineTuned_Final',output_hidden_states = True,)                    #posttrained
#model = BertForSequenceClassification.from_pretrained('Desktop/SavedModel',output_hidden_states = True,) #finetuned

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
print("BERT model is gedownload.")

In [ ]:
#Create pretrained vectors
start = time.time()
bert_vectors = []
review_counter = 0
#loops over the reviews
for rev in reviews:
    result = []
    tokenized_text = tokenizer.tokenize(rev)
    #change to 512 or shorter
    if len(tokenized_text)>=512:
        del tokenized_text[512:len(tokenized_text)]
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs.hidden_states
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    token_vecs_sum = []
    for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec)
    token_vecs = hidden_states[-2][0]
    sentence_embedding = torch.mean(token_vecs, dim=0)
    #get all results for one review and display
    # replace with whole vector!
    i=0
    while i < len(tokenized_text):
        veccie = [round(vec,4) for vec in token_vecs_sum[i].tolist()]
        result.append([tokenized_text[i],indexed_tokens[i],segments_ids[i], veccie, review_counter])
        i += 1
    bert_vectors.append(result)
    del result
    review_counter+=1
end = time.time()
print("Tijd: ", end-start, "s")
print("Reviews zijn getokenized en omgezet naar vectoren!")

In [ ]:
#Load weights for fine-tuned model here to create word embeddings with fine-tuned model
model = BertModel.from_pretrained('Desktop/BERT_FineTuned_Final',output_hidden_states = True,)                    #posttrained
model.eval()
print("BERT Fine-Tuned model is gedownload.")

In [ ]:
#Creates finetuned vectors
start = time.time()
bert_vectors_ft = []
review_counter = 0
#loops over the reviews
for rev in reviews:
    result = []
    tokenized_text = tokenizer.tokenize(rev)
    #change to 512 or shorter
    if len(tokenized_text)>=512:
        del tokenized_text[512:len(tokenized_text)]
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs.hidden_states
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    token_vecs_sum = []
    for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec)
    token_vecs = hidden_states[-2][0]
    sentence_embedding = torch.mean(token_vecs, dim=0)
    #get all results for one review and display
    # replace with whole vector!
    i=0
    while i < len(tokenized_text):
        veccie = [round(vec,4) for vec in token_vecs_sum[i].tolist()]
        result.append([tokenized_text[i],indexed_tokens[i],segments_ids[i], veccie, review_counter])
        i += 1
    bert_vectors_ft.append(result)
    del result
    review_counter+=1
end = time.time()
print("Tijd: ", end-start, "s")
print("Reviews zijn getokenized en omgezet naar vectoren!")

In [ ]:
#Create good format for ontology
#Change list1 to the location of bert vectors
start = time.time()
list1 = bert_vectors_ft
words = {}
j=1
counterRev = 0
for rev in list1:
    counterRev += 1
    for word in rev:
        string1 = str(word[0])
        words[j] = {'word': string1,
                    'vector': word[3]
                    ,'sentence id': word[4]
                   }
        j = j+1
        
print('Number of reviews done: ', counterRev, ', with ', j, ' words in total.')

In [ ]:
#Save file
import json
start = time.time()
name_file = "file.txt"
with open(name_file, 'w') as outfile:
    json.dump(words, outfile)
end = time.time()
print("Tijd: ", end-start, "s")
print("Alles staat in " + name_file)